In [ ]:
import pandas as pd
import numpy as np

import os.path as op
import sys

sys.path.append(op.abspath('../code'))

import plotting_tools

import importlib

import data_loaders as dload
import clusterize as cluster

import ipywidgets

In [ ]:
default_path = '/mnt/c/Users/Cionkito/Documents/Brainhack2022/Piriform_neurons'

path_to_Piriform_neurons_widg = ipywidgets.Textarea(value=default_path,
                                                placeholder='Insert your path here',
                                                description='String:',
                                                disabled=False)
display(path_to_Piriform_neurons_widg)

In [ ]:
#path_to_Piriform_neurons = '/mnt/c/Users/Cionkito/Documents/Brainhack2022/Piriform_neurons'
importlib.reload(dload)
path_to_Piriform_neurons = path_to_Piriform_neurons_widg.value
path_to_xls = op.join(path_to_Piriform_neurons, 'data')
neuron_df = dload.get_neuron_matrix(path_to_xls, verb = False)

is_na_idx = np.where(neuron_df.isna())[0]

cluster_df = neuron_df.dropna(axis = 0).apply(cluster.normalize).dropna(axis = 1)

In [ ]:
feature_selection =  ipywidgets.SelectMultiple(options=cluster_df.columns.tolist(),
                                                value=[],
                                                rows=len(cluster_df.columns.tolist())+1,
                                                description='Features',
                                                description_width = 20,
                                                disabled=False)

#feature_selection_tag = ipywidgets.TagsInput(value=cluster_df.columns.tolist(),
#                                            allowed_tags=cluster_df.columns.tolist(),
#                                            allow_duplicates=False)

### Feature selection
Here you may select the features you wish to keep for the clustering. To do so, please, hold `CMD` (or `CTRL` for Windows users) and select the desired features. You can select all feature using `SHIFT` and sliding from the top to the bottom of the list.

In [ ]:
display(feature_selection)

In [ ]:
#is_na_idx = np.where(my_matrix.isna())[0]
#
#cluster_df = my_matrix.dropna(axis = 0).apply(cluster.normalize).dropna(axis = 1)
#
#feat_to_del = ['Filament BoundingBoxAA Length X', 'Filament BoundingBoxAA Length Y', 'Filament BoundingBoxAA Length Z']
#cluster_df = cluster_df.drop(columns = feat_to_del)

cluster_filtered_df = cluster_df.loc[:, feature_selection.value].copy()

print(f' {cluster_filtered_df.shape[1]} columns selected for the clustering :\n-',
      '\n- '.join(cluster_filtered_df.columns.tolist()))
print(f'\n{cluster_filtered_df.shape[0]} neurons selected for the clustering :\n',
      cluster_filtered_df.index.tolist())

#cluster_df

In [ ]:
show_figures = ipywidgets.ToggleButton(value=False,
                    description='Show figures',
                    disabled=False,
                    indent=False)

save_figures = ipywidgets.ToggleButton(value=False,
                    description='Save figures',
                    disabled=False,
                    indent=False)

### Figure parameters
Here you can specify if you wish to show and save the clustering figures by pressing the respective butttons.

In [ ]:
ipywidgets.Box([show_figures, save_figures])

In [ ]:
importlib.reload(cluster)

cluster_model, label_2_color = cluster.hierarchical_cstm(cluster_df, d_thresh = 11.9,
                                                         figsize = (30, 10), plot = show_figures.value)

labels = cluster_model.labels_
n_clusters = labels.max() + 1

if save_figures.value:
    plotting_tools.save_figure(op.join(path_to_Piriform_neurons,
                                       f'figures/hierarchical-tree-{n_clusters:02d}clusters.png'))

centroids = cluster.get_centroids(cluster_model, cluster_df, col = label_2_color,
                                  plot = show_figures.value)

if save_figures.value:
    plotting_tools.save_figure(op.join(path_to_Piriform_neurons,
                                       f'figures/hierarchical-centroids-{n_clusters:02d}clusters.png'))

dist_to_centroid = cluster.get_centroid_distance(cluster_model, cluster_df, centroids)

### Interpretation of clustering results (above)
- The clustering stability (in distance) shows the distance before two clusters are merged into one. In other words, the larger the distance, the more stable is the clustering for a specified  number of clusters _k_.
- The number of clusters is only given if the clustering is parametrized through a specified distance.
- The _Groups_ shows the number of samples (neurons) in each cluster.

In [ ]:
#print('List of neurons in each cluster:')
#
#for i in range(n_clusters):
#    print(f'cluster #{i + 1} :', cluster_df.loc[labels == i].index.tolist())

label_save_df = cluster_df.iloc[:, :1].copy()
label_save_df[f'labels-{n_clusters}k'] = labels

#label_save_df = label_save_df.drop(columns = ['Filament BoundingBoxOO Length A'])

# Un-comment to save cluster labels (from 0 to n_cluster-1) in .csv file
# where each row is a neuron with an ID and its corresponding label

#label_save_df.to_csv('/Users/AlexCionca/Documents/BrainHack-2023/Piriform_neurons/neuron_clustering.csv')

### List of neurons in each cluster :

In [ ]:
for i in range(n_clusters):
    print(f'cluster #{i + 1} :', cluster_df.loc[labels == i].index.tolist())

### Features with the least in-cluster variability  (most in-cluster similarity) :

In [ ]:
for i, cluster_features in enumerate(dist_to_centroid.feat_by_radius):
    print(f'cluster #{i + 1:d}', cluster_features[:4])